In [1]:
# import packages
import matplotlib.pyplot as plt
import numpy as np
import cv2

from scipy.spatial import Delaunay

#### Function: read data from file

In [2]:
"""
    Data type:
        path_id
        point_type
        x_rel
        y_rel
        point_id
        connectsFrom
        connectsTo
    Return:
        path_id
        point_type
        pointSet = (x_rel, y_rel)
        point_id
        connectsFrom
        connectsTo
"""
# read data from file
def get_data(file_name):
    with open(file_name) as f:
        # fist line is the number of points
        line = f.readline()
        num_points = int(line)
        #create several np.array to store the data
        path_id = np.zeros(num_points)
        point_type = np.zeros(num_points)
        x_rel = np.zeros(num_points)
        y_rel = np.zeros(num_points)
        pointSet = np.zeros((num_points, 2))
        point_id = np.zeros(num_points)
        connectsFrom = np.zeros(num_points)
        connectsTo = np.zeros(num_points)

        #read data
        for i in range(0, num_points):
            line = f.readline().split(' ')
            path_id[i] = int(line[0])
            point_type[i] = int(line[1])
            x_rel[i] = float(line[2]) * 640
            y_rel[i] = float(line[3]) * 480
            pointSet[i] = np.array([x_rel[i], y_rel[i]])
            point_id[i] = int(line[4])
            connectsFrom[i] = int(line[5])
            connectsTo[i] = int(line[6])

    return path_id, point_type, pointSet, point_id, connectsFrom, connectsTo

#=====================================================================
"""
Slight difference in pixel ordering in OpenCV and Matplotlib.
OpenCV follows BGR order, while matplotlib likely follows RGB order.
So the image need to re-order to show correctly
"""
def reordering(image):
    b1, g1, r1 = cv2.split(image)
    # re-order
    reorder = cv2.merge([r1, g1, b1])
    
    return reorder

#=====================================================================
"""
Alpha blending, combine two images
imput: src_image ===> source image
       dst_image ===> destination image
       alpha     ===> rate
return: image = alpha * src_image + (1 - alpha) * dst_image
"""
def combining(srd_image, dst_image, alpha):
    # split into b, g, r three channels
    b1, g1, r1 = cv2.split(srd_image)
    b2, g2, r2 = cv2.split(dst_image)
    # combine
    beta = 1 - alpha
    b_final = cv2.multiply(alpha, b1) + cv2.multiply(beta, b2)
    g_final = cv2.multiply(alpha, g1) + cv2.multiply(beta, g2)
    r_final = cv2.multiply(alpha, r1) + cv2.multiply(beta, r2)
    # merge
    final = cv2.merge([b_final, g_final, r_final])
    
    return final

#=====================================================================
"""
create a mask according to the given coordinates
input: polygon_points ===> coordinates
       image          ===> original image
output: mask with the same size of the image
"""
def mask(polygon_points, image):
    #image = image.reshape(-1)
    # mask defaulting to black for 3-channel and transparent for 4-channel
    # (of course replace corners with yours)
    mask = np.zeros(image.shape, dtype=np.uint8)
    #roi_corners = np.array([[(10,10), (300,300), (10,300)]], dtype=np.int32)
    # fill the ROI so it doesn't get wiped out when the mask is applied
    channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,) * channel_count
    cv2.fillConvexPoly(mask, polygon_points.astype(dtype='int32'), ignore_mask_color)
    # from Masterfool: use cv2.fillConvexPoly if you know it's convex

    # apply the mask
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

#=====================================================================
"""
create a mask_inverse according to the given coordinates
input: polygon_points ===> coordinates
       image          ===> original image
output: mask with the same size of the image
"""
def mask_img(polygon_points, image):
    #image = image.reshape(-1)
    # mask defaulting to black for 3-channel and transparent for 4-channel
    # (of course replace corners with yours)
    mask = np.zeros(image.shape, dtype=np.uint8)
    #roi_corners = np.array([[(10,10), (300,300), (10,300)]], dtype=np.int32)
    # fill the ROI so it doesn't get wiped out when the mask is applied
    channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,) * channel_count
    cv2.fillConvexPoly(mask, polygon_points.astype(dtype='int32'), ignore_mask_color)
    
    return mask

#=====================================================================
"""
Transparency: image * alpha, alpha = [0, 1]
input: image, alpha
outut: image * alpha
"""
def img_mul(image, alpha):
    # split the image
    b, g, r = cv2.split(image)
    # multiplication
    b_final = cv2.multiply(alpha, b)
    g_final = cv2.multiply(alpha, g)
    r_final = cv2.multiply(alpha, r)
    # merge
    final = cv2.merge([b_final, g_final, r_final])
    
    return final

def generate_face(face, image, roi):
    # I want to put logo on top-left corner, So I create a ROI
    rows, cols, channels = img2.shape

    # Now create a mask of logo and create its inverse mask
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

    # add a threshold
    ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)

    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

    dst = cv2.add(img1_bg,img2_fg)
    img1[0:rows, 0:cols ] = dst

#=====================================================================
"""
Face morphing
"""
def morphing(src, dst, mid_img, tri_src, tri_dst, tri_mid, alpha):
    # Find bounding rectangle points for each triangle
    # to prevent gaps
    # cv2.boundingRect return [x, y, w, h]
    # (x,y) be the top-left coordinate of the rectangle and (w,h) be its width and height.
    rec_src = cv2.boundingRect(tri_src.astype(dtype='float32'))
    rec_dst = cv2.boundingRect(tri_dst.astype(dtype='float32'))
    rec_mid = cv2.boundingRect(tri_mid.astype(dtype='float32'))
    # used for cropping the mask
    src_rect = []
    dst_rect = []
    mid_rect = []
    for i in range(0, 3):
        src_rect.append(((tri_src[i, 0] - rec_src[0]),(tri_src[i, 1] - rec_src[1])))
        dst_rect.append(((tri_dst[i, 0] - rec_dst[0]),(tri_dst[i, 1] - rec_dst[1])))
        mid_rect.append(((tri_mid[i, 0] - rec_mid[0]),(tri_mid[i, 1] - rec_mid[1])))
    # mask of the rectangle
    #mask = mask_img(tri_src, np.zeros([rec_mid_pts[3], rec_mid_pts[2], 3]))
    mask = np.zeros((rec_mid[3], rec_mid[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(mid_rect), (1.0, 1.0, 1.0), 16, 0);
    
    # warping
    # src / dst to mid
    #src_rect_index = 
    #dst_rect_index = np.array()
    src_rect_pts = src[rec_src[1]:rec_src[1] + rec_src[3], rec_src[0]:rec_src[0] + rec_src[2]]
    dst_rect_pts = dst[rec_dst[1]:rec_dst[1] + rec_dst[3], rec_dst[0]:rec_dst[0] + rec_dst[2]]
    # warp size = [rec_mid_w, rec_mid_h], because from src/dst to mid
    warp_size = (rec_mid[2], rec_mid[3])
    # warping matrix
    src_warp_mat = cv2.getAffineTransform(np.float32(src_rect), np.float32(mid_rect))
    dst_warp_mat = cv2.getAffineTransform(np.float32(dst_rect), np.float32(mid_rect))
    # warping image
    warp_src = cv2.warpAffine(src_rect_pts, src_warp_mat, (warp_size[0], warp_size[1]))
    warp_dst = cv2.warpAffine(dst_rect_pts, dst_warp_mat, (warp_size[0], warp_size[1]))
    # alpha blending
    mid_img_rect = combining(warp_src, warp_dst, alpha)
    # save the mask to the mid image
    mid_rect_index = np.array([])
    mid_img[rec_mid[1]:rec_mid[1] + rec_mid[3], rec_mid[0]:rec_mid[0] + rec_mid[2]] = \
    mid_img[rec_mid[1]:rec_mid[1] + rec_mid[3], rec_mid[0]:rec_mid[0] + rec_mid[2]] * (1 - mask) + mid_img_rect * mask
    

In [3]:

def main(alpha = 0.5):
    #file name
    face1_file = "./face_data/01-1m.bmp"
    face1_data_file = "./face_data/01-1m.txt"
    face2_file = "./face_data/05-1m.bmp"
    face2_data_file = "./face_data/05-1m.txt"

    # set figure size
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 24
    fig_size[1] = 16


    # read data
    """
    Data type:
        path_id
        point_type
        x_rel
        y_rel
        point_id
        connectsFrom
        connectsTo
    """
    path_id1, point_type1, pointSet1, point_id1, connectsFrom1, connectsTo1 = get_data(face1_data_file)
    path_id2, point_type2, pointSet2, point_id2, connectsFrom2, connectsTo2 = get_data(face2_data_file)
    
    # num of points
    num_pts = len(path_id1)

    # Delaunay triangulation
    pointSet1 = pointSet1.astype(dtype='int32')
    pointSet2 = pointSet2.astype(dtype='int32')
    del_triangle1 = Delaunay(pointSet1)
    del_triangle2 = Delaunay(pointSet2)

    # plot the triangles
    #plt.triplot(pointSet1[:,0], pointSet1[:,1], del_triangle1.simplices.copy())

    # read image file
    img1 = cv2.imread(face1_file)
    img2 = cv2.imread(face2_file)
    img1 = img1.astype(dtype='float32')
    img2 = img2.astype(dtype='float32')
    print (type(img1[1, 1, 1]))

    """
    Slight difference in pixel ordering in OpenCV and Matplotlib.
    OpenCV follows BGR order, while matplotlib likely follows RGB order.
    So the image need to re-order to show correctly
    When using cv2.imshow, no need to re-order
    """
    
    # image in between
    # point_mid = alpha * point_in_src + (1 - alpha) * point_in_dst
    pointSet_mid = []
    beta = 1 - alpha
    for i in range(num_pts):
        x_mid = alpha * pointSet1[i, 0] + beta * pointSet2[i, 0]
        y_mid = alpha * pointSet1[i, 1] + beta * pointSet2[i, 1]
        pointSet_mid.append((x_mid, y_mid))
    pointSet_mid = np.asarray(pointSet_mid)
    print (type(pointSet_mid))
    print (pointSet_mid.shape)
    
    # morphing image
    # create an empty image
    img_mid = np.zeros(img1.shape, dtype = img1.dtype)
    
    # morphing: one triangle each time
    for tri in del_triangle1.simplices:
        # get the coordinates from the source image
        tri_src = pointSet1[tri].astype(dtype='float32')
        # get the coordinates from the destination image
        tri_dst = pointSet2[tri].astype(dtype='float32')
        # get the coordinates from the destination image
        tri_mid = pointSet_mid[tri].astype(dtype='float32')        
        morphing(img1, img2, img_mid, tri_src, tri_dst, tri_mid, alpha)
    
    # =====================================================================
    """
    Display the image with matplotlib
    Something wrong when display the image from opencv
    Not used anymore.   
    """
    #f, (ax1, ax2) = plt.subplots(1, 2, sharex='all', sharey='all')
    #plt.subplot(121), plt.imshow(img1), plt.title('Src')
    #plt.scatter([270, 400], [150, 150])
    #face1 = mask_tri1 + face1
    #plt.subplot(122), plt.imshow(img_mid), plt.title('In-between')
    print (type(pointSet1[1, 1]))
    #plt.scatter(pointSet1[:, 0], pointSet1[:, 1])
    #plt.triplot(pointSet1[:,0], pointSet1[:,1], del_triangle1.simplices.copy())
    #plt.show()
    #plt.show()
    #rec_src = cv2.boundingRect(pointSet1[tri].astype(dtype='float32'))
    #print (rec_src)
    # =====================================================================

    cv2.imshow("Morphed Face", np.uint8(img_mid))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

main()

<class 'numpy.float32'>
<class 'numpy.ndarray'>
(58, 2)
<class 'numpy.int32'>


In [19]:
from os import listdir
import os

def read_asf(file_name):
    #create several lists to store the data
    path_id = []
    point_type = []
    x_rel = []
    y_rel = []
    pointSet = []
    point_id = []
    connectsFrom = []
    connectsTo = []
    with open(file_name) as f:
        for line in f:
            input_line = line.strip()
            # ignore comments
            if not input_line.startswith('#'):
                # first line shows the number of points, 2 chars
                if len(input_line) > 0 and len(input_line) < 5:
                    num_points = int(input_line)

                #read data
                elif len(input_line) > 10:
                    str_split = input_line.split(' ')
                    path_id.append(int(str_split[0]))
                    point_type.append(int(str_split[1]))
                    x_rel.append(float(str_split[2]) * 640)
                    y_rel.append(float(str_split[3]) * 480)
                    pointSet.append(np.array([float(str_split[2]) * 640, float(str_split[3]) * 480]))
                    point_id.append(int(str_split[4]))
                    connectsFrom.append(int(str_split[5]))
                    connectsTo.append((str_split[6]))

    return path_id, point_type, pointSet, point_id, connectsFrom, connectsTo

path_id, point_type, pointSet, point_id, connectsFrom, connectsTo = read_asf("./face_data/01-1m.asf")
print (np.asarray(pointSet).shape)

def mean_face_shape(dir):
    # initialize
    mean_shape = np.zeros([58, 2])
    # counting the number of files
    count = 0
    # scan all the file names in dir
    for file in listdir(dir):
        # "*.asf" file is what we need
        if (".asf" in file):
            # read points
            path_id, point_type, pointSet, point_id, connectsFrom, connectsTo = read_asf(dir + file)
            # compute the mean face
            mean_shape = pointSet + mean_shape
            # count increment
            count = count + 1
    
    # average
    mean_shape = mean_shape / count
    return mean_shape

mean_shape = mean_face_shape("./face_data/data/")
type(mean_shape)


(58, 2)


numpy.ndarray

In [14]:
def mean_face(dir):
    # initialize
    mean_face = np.zeros([480, 640, 3])
    # counting the number of files
    count = 0
    # scan all the file names in dir
    for file in listdir(dir):
        # "*.asf" file is what we need
        if (".bmp" in file):
            # read image
            face_img = cv2.imread(dir + file)
            # compute the mean face
            mean_face = np.float32(face_img) + mean_face
            count = count + 1
    print (count)
    # average
    mean_face = mean_face / count
    return mean_face

mean_face = mean_face("./face_data/data/")
print (mean_face)
#mean_face = cv2.imread("./face_data/data/01-1m.bmp")
cv2.imshow("Morphed Face", np.uint8(mean_face))
cv2.waitKey(0)
cv2.destroyAllWindows()

37
[[[ 50.35135135  45.72972973  30.75675676]
  [ 48.18918919  43.32432432  28.18918919]
  [ 48.97297297  43.97297297  26.91891892]
  ..., 
  [ 42.75675676  39.78378378  23.51351351]
  [ 42.97297297  39.72972973  23.35135135]
  [ 42.97297297  39.72972973  23.35135135]]

 [[ 49.40540541  46.81081081  30.62162162]
  [ 45.51351351  42.56756757  26.27027027]
  [ 47.          43.83783784  25.64864865]
  ..., 
  [ 42.10810811  40.02702703  22.78378378]
  [ 41.43243243  38.89189189  21.45945946]
  [ 41.05405405  38.51351351  21.08108108]]

 [[ 50.05405405  47.13513514  30.24324324]
  [ 46.59459459  43.37837838  26.35135135]
  [ 47.56756757  44.45945946  25.05405405]
  ..., 
  [ 41.75675676  40.18918919  22.05405405]
  [ 41.45945946  39.59459459  21.43243243]
  [ 41.86486486  40.          21.83783784]]

 ..., 
 [[ 40.75675676  38.89189189  24.64864865]
  [ 38.62162162  36.64864865  22.18918919]
  [ 40.13513514  37.72972973  21.81081081]
  ..., 
  [ 43.13513514  44.81081081  23.81081081]
  [ 42